# Begin any necessary cleaning adjustments and transforming data into a format to better utilize in Tableau

In [1]:
#import libraries and mount google drive
import pandas as pd
import numpy as np
import plotly.express as px
from google.colab import drive
#mount google drive, input verification code
drive.mount('/content/drive')

Mounted at /content/drive


### Import data and check to make sure everything imported correctly

#### Well, looks like I imported all department numbers and not just the ones I wanted. Lets check that and also check the operation_type Column to make sure only SALE was imported.


In [9]:
df= pd.read_csv('/content/drive/MyDrive/PersonalProfessional/Weekly Sales Project/CLEAN_DATA_FOR_PYTHON.csv')
df.head()

,date,operation_type,store_code,department,quantity
0,4/30/2021,SALE,400000038360,Dep1,5
1,4/30/2021,SALE,400000038155,Dep2,1
2,4/30/2021,SALE,400000014128,Dep1,1
3,4/30/2021,SALE,400000038162,Dep2,1
4,4/30/2021,SALE,400000034003,Dep3,1


In [11]:
print(df['operation_type'].value_counts())
print(df['department'].value_counts())

SALE    51094
Name: operation_type, dtype: int64
Dep2     16198
Dep1     14935
Dep5     10758
Dep12     1629
Dep10     1557
Dep7       869
Dep20      696
Dep3       686
Dep15      592
Dep8       578
Dep9       484
Dep17      374
Dep13      362
Dep21      353
Dep11      272
Dep6       192
Dep4       172
Dep22       88
Dep18       69
Dep14       52
Dep25       50
Dep23       48
Dep16       44
Dep24       14
Dep26       10
Dep27        7
Dep19        5
Name: department, dtype: int64


### Ok, we have to clean up departments some more, but since operation_type was cleaned correctly (and isn't needed going further) we'll drop that column

In [12]:
df.drop('operation_type',axis=1, inplace=True)
df.head()

,date,store_code,department,quantity
0,4/30/2021,400000038360,Dep1,5
1,4/30/2021,400000038155,Dep2,1
2,4/30/2021,400000014128,Dep1,1
3,4/30/2021,400000038162,Dep2,1
4,4/30/2021,400000034003,Dep3,1


### Before we mess with the departments mistake, lets make sure the data types are what we want

In [15]:
df.dtypes

date          object
store_code     int64
department    object
quantity       int64
dtype: object

### Looks good.

### I want to group the sales by week to find how many products we sell in a weekly basis. So we must turn the date columns into datetime

In [16]:
df['date'] = pd.to_datetime(df['date'])
df.dtypes

date          datetime64[ns]
store_code             int64
department            object
quantity               int64
dtype: object

### I wanted to use week numbers, but since this data goes over multiples years, I will extract the year and the week number then join them back together to give an ID to the week-year

In [18]:
#Create new columns for week number and year
df['week']=df.date.dt.isocalendar().week
df['year']=df.date.dt.year
df.head()

,date,store_code,department,quantity,week,year
0,2021-04-30,400000038360,Dep1,5,17,2021
1,2021-04-30,400000038155,Dep2,1,17,2021
2,2021-04-30,400000014128,Dep1,1,17,2021
3,2021-04-30,400000038162,Dep2,1,17,2021
4,2021-04-30,400000034003,Dep3,1,17,2021


In [20]:
#join those columns together
df['week_number']=df[['week', 'year']].astype('str').agg('-'.join, axis=1)


,date,store_code,department,quantity,week,year,week_number
0,2021-04-30,400000038360,Dep1,5,17,2021,17-2021
1,2021-04-30,400000038155,Dep2,1,17,2021,17-2021
2,2021-04-30,400000014128,Dep1,1,17,2021,17-2021
3,2021-04-30,400000038162,Dep2,1,17,2021,17-2021
4,2021-04-30,400000034003,Dep3,1,17,2021,17-2021


In [29]:
#sort values from oldest date nearest
df.sort_values('date', inplace=True)
df.head()

,date,store_code,department,quantity,week,year,week_number
51093,2019-01-01,400000012018,Dep5,1,1,2019,1-2019
51075,2019-01-02,400000011394,Dep5,1,1,2019,1-2019
51076,2019-01-02,400000004389,Dep5,1,1,2019,1-2019
51077,2019-01-02,400000006727,Dep5,1,1,2019,1-2019
51078,2019-01-02,400000016702,Dep5,1,1,2019,1-2019


### Cool! Lets isolate only rows with Dep1 and Dep2 and save each of those departments to their own dataframes, 'dep1', and 'dep2'

In [30]:
dep1 = df[(df['department'] == 'Dep1')].reset_index()
dep2 = df[(df['department'] == 'Dep2')].reset_index()

In [25]:
#Check to make sure data types are preserved and shape is as expected.
print(dep1.dtypes)
print(dep1.shape)

index                   int64
date           datetime64[ns]
store_code              int64
department             object
quantity                int64
week                   UInt32
year                    int64
week_number            object
dtype: object
(14935, 8)


### Seems good!

### Now that we have the departments in their own dataframes and we have cleaned up the data a bit more, we have some essential goals

1. Drop 'week' and 'year' column
2. Group by week and sum quantities for each store_code
3. Turn data into pivot tables for further analysis

### Lets begin with department 1

In [38]:
# Groupes by week_number, and store_code giving the sum for each store_code by week_number
# Note: This does not preserve the order, so running certain time related analysis will not work
# For our case this will be fine
dep1_grouped=dep1.groupby(['week_number', 'store_code']).agg({'quantity':'sum'})

In [39]:
dep1_grouped.head()

quantity
week_number store_code            
1-2019      400000000480         5
            400000000497         1
            400000000916         1
            400000001050         2
            400000001401         1

In [46]:
dep1_pivot=dep1_grouped.pivot_table(values='quantity', index='store_code', columns='week_number')
dep1_pivot

week_number,1-2019,1-2020,1-2021,10-2019,10-2020,10-2021,11-2019,11-2020,11-2021,12-2019,12-2020,12-2021,13-2019,13-2020,13-2021,14-2019,14-2020,14-2021,15-2019,15-2020,15-2021,16-2019,16-2020,16-2021,17-2019,17-2020,17-2021,18-2019,18-2020,19-2019,19-2020,2-2020,2-2021,20-2019,20-2020,21-2019,21-2020,22-2019,22-2020,23-2019,...,4-2021,40-2019,40-2020,41-2019,41-2020,42-2019,42-2020,43-2019,43-2020,44-2019,44-2020,45-2019,45-2020,46-2019,46-2020,47-2019,47-2020,48-2019,48-2020,49-2019,49-2020,5-2020,5-2021,50-2019,50-2020,51-2019,51-2020,52-2019,52-2020,53-2020,53-2021,6-2020,6-2021,7-2020,7-2021,8-2020,8-2021,9-2019,9-2020,9-2021
store_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
400000000329,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000000343,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000000350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
400000000367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000000480,5.0,3.0,8.0,NaN,9.0,9.0,NaN,3.0,14.0,NaN,16.0,10.0,NaN,6.0,18.0,NaN,10.0,17.0,5.0,19.0,16.0,2.0,19.0,14.0,NaN,30.0,6.0,1.0,37.0,1.0,24.0,5.0,22.0,NaN,13.0,NaN,24.0,NaN,18.0,NaN,...,20.0,4.0,16.0,5.0,27.0,4.0,16.0,4.0,16.0,2.0,11.0,2.0,15.0,NaN,2.0,NaN,13.0,1.0,14.0,1.0,17.0,6.0,7.0,9.0,5.0,7.0,9.0,3.0,10.0,28.0,23.0,10.0,15.0,8.0,12.0,5.0,14.0,3.0,7.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400000038322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000038353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000038360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Awesome! Lets repeat this process for dep2

#### Quick Note:
        I am choosing to leave the null values for these calculations. It helps when dealing with products that may be very new, or during 2020 when certain products were hard to come by. 
        
         Again, the goal of this analysis is to understand product movement when we have the product. I felt like this was the best choice.

In [41]:
dep2_grouped=dep2.groupby(['week_number', 'store_code']).agg({'quantity':'sum'})
dep2_pivot=dep2_grouped.pivot_table(values='quantity', index='store_code', columns='week_number')
dep2_pivot

week_number,1-2019,1-2020,1-2021,10-2019,10-2020,10-2021,11-2019,11-2020,11-2021,12-2019,12-2020,12-2021,13-2019,13-2020,13-2021,14-2019,14-2020,14-2021,15-2019,15-2020,15-2021,16-2019,16-2020,16-2021,17-2019,17-2020,17-2021,18-2019,18-2020,19-2019,19-2020,2-2019,2-2020,2-2021,20-2019,20-2020,21-2019,21-2020,22-2019,22-2020,...,41-2020,42-2019,42-2020,43-2019,43-2020,44-2019,44-2020,45-2019,45-2020,46-2019,46-2020,47-2019,47-2020,48-2019,48-2020,49-2019,49-2020,5-2019,5-2020,5-2021,50-2019,50-2020,51-2019,51-2020,52-2019,52-2020,53-2020,53-2021,6-2019,6-2020,6-2021,7-2019,7-2020,7-2021,8-2019,8-2020,8-2021,9-2019,9-2020,9-2021
store_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
400000014302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,2.0,NaN,NaN,NaN,2.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,NaN,1.0,NaN,2.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
400000015705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000015729,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000015736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000016214,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400000038414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000038575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400000038599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Using the data that we have accumulated into intese pivot tables, lets get some statistics so we can view everything a little easier
### We can also use this 'stats' dataframe to creat some charts that will help us determine how often these products sell and overall how popular the products are.


Stats for department 1

In [68]:

dep1_stats=pd.DataFrame(index=dep1_pivot.index)
dep1_stats['total_sold']=dep1_pivot.sum(axis=1)
dep1_stats['average']=dep1_pivot.mean(axis=1)
dep1_stats

,total_sold,average
store_code,,
400000000329,20.0,2.222222
400000000343,37.0,2.846154
400000000350,14.0,1.555556
400000000367,22.0,3.666667
400000000480,1263.0,13.156250
...,...,...
400000038322,1.0,1.000000
400000038353,3.0,1.000000
400000038360,15.0,7.500000


Stats for department 2

Lets Make sure data types are where we want them to be

In [69]:
dep2_stats=pd.DataFrame(index=dep2_pivot.index)
dep2_stats['total_sold']=dep2_pivot.sum(axis=1)
dep2_stats['average']=dep2_pivot.mean(axis=1)
dep2_stats

,total_sold,average
store_code,,
400000014302,53.0,1.606061
400000015705,2.0,1.000000
400000015729,8.0,2.666667
400000015736,2.0,1.000000
400000016214,16.0,1.142857
...,...,...
400000038414,1.0,1.000000
400000038575,1.0,1.000000
400000038599,4.0,2.000000


In [70]:
dep1_stats.reset_index(inplace=True)
dep2_stats.reset_index(inplace=True)
print(dep1_stats.dtypes)
print(dep2_stats.dtypes)

store_code      int64
total_sold    float64
average       float64
dtype: object
store_code      int64
total_sold    float64
average       float64
dtype: object


We want store_code to be a string, lets fix that

In [84]:
dep1_stats['store_code']= dep1_stats['store_code'].astype('str')
dep2_stats['store_code']= dep2_stats['store_code'].astype('str')
print(dep1_stats.dtypes)
print(dep2_stats.dtypes)

store_code     object
total_sold    float64
average       float64
dtype: object
store_code     object
total_sold    float64
average       float64
dtype: object


I also want to shorten the Store_code being that the first bits are repeated throughout, meaning only the last 5 digits are needed

Lets slice off those unneeded numbers and store it as 'short_code'

In [87]:
dep1_stats['short_code']=dep1_stats['store_code'].str[7:]
dep2_stats['short_code']=dep2_stats['store_code'].str[7:]
dep1_stats

,store_code,total_sold,average,short_code
0,400000000329,20.0,2.222222,00329
1,400000000343,37.0,2.846154,00343
2,400000000350,14.0,1.555556,00350
3,400000000367,22.0,3.666667,00367
4,400000000480,1263.0,13.156250,00480
...,...,...,...,...
376,400000038322,1.0,1.000000,38322
377,400000038353,3.0,1.000000,38353
378,400000038360,15.0,7.500000,38360
379,400000038827,3.0,3.000000,38827


In [94]:
#Plotly handles strings of numbers wierdly, we are gonna add a P to the beginning of the short code
dep1_stats['short_code'] = 'P' + dep1_stats['short_code'].astype(str)
dep2_stats['short_code'] = 'P' + dep2_stats['short_code'].astype(str)

## Now we can take these stats tables to Tableau, but before we do lets see some quick visualizations in Python using Plotly.
## I enjoy plotly because of the built-in interactivity

In [106]:
from plotly.subplots import make_subplots #Visualizations
import plotly.graph_objects as go #visualizations
#Visualizations using Plotly subplots

#Sort Values ascending = False on total_sold and grab top 20 for visualization
dep1_top_20=dep1_stats.sort_values('total_sold', ascending=False).head(20).reset_index()
dep2_top_20=dep2_stats.sort_values('total_sold', ascending=False).head(20).reset_index()

#Sort values ascending = True and grab 20 worst sellers
dep1_bottom_20=dep1_stats.sort_values('total_sold', ascending=True).head(20).reset_index()
dep2_bottom_20=dep2_stats.sort_values('total_sold', ascending=True).head(20).reset_index()



fig= make_subplots(rows=4, cols=2, subplot_titles=( 'Count of Averages for Units Sold in Dep1', 'Count of Averages for Units Sold in Dep1','Dept 1 Top 20', 
'Dept 2 Top 20','','','Dept 1 Bottom 20', 'Dept 2 Bottom 20'))

fig.add_trace(go.Histogram(x=dep1_stats['average']),row=1,col=1)#Distribution to see average sold per week for dept 1
fig.add_trace(go.Histogram(x=dep2_stats['average']),row=1,col=2)#Distribution to see average sold per week for dept 2
fig.add_trace(go.Bar(x=dep1_top_20['short_code'], y=dep1_top_20['total_sold']), row=2, col=1) #Dept 1 top 20 sellers by total
fig.add_trace(go.Bar(x=dep2_top_20['short_code'], y=dep2_top_20['total_sold']), row=2, col=2) #Dept 2 top 20 sellers by total
fig.add_trace(go.Bar(x=dep1_bottom_20['short_code'], y=dep1_bottom_20['total_sold']), row=4, col=1) #Dept 1 bottom 20 sellers by total
fig.add_trace(go.Bar(x=dep2_bottom_20['short_code'], y=dep2_bottom_20['total_sold']), row=4, col=2) #Dept 2 bottom 20 sellers by total

fig.update_layout(showlegend=False, title_text="Summary Graphs")

Some insights. 

We have the top and bottom performing products for the last 28 months. 
We can also see that we sell, on average 1-2 products per week from these categories!